In [2]:
import json
import time
import uuid
import requests
from IPython.display import display, Markdown

## Test embedding, chat

In [3]:
import requests
import time
import uuid

def get_answer(question, mem_id=None):
    id_ = uuid.uuid4().hex
    if not mem_id:
        payload = {
            "id": id_,
            "question": question,
        }
    else:
        payload = {
            "id": id_,
            "question": question,
            "memory_id": mem_id,
        }

    url = "http://localhost:8000/chat"
    r = requests.post(url, json=payload)
    r.raise_for_status()
    r = r.json()
    mem_id = r["memory_id"]
    print("Request sent. Waiting for response...")
    time.sleep(10)

    num_retries = 0
    while True:
        try:
            url = f"http://localhost:8000/get/{id_}"
            r = requests.get(url)
            try:
                r.raise_for_status()
                r = r.json()
                if r[0]["processed"]:
                    break
                print("Waiting for response...")
                time.sleep(3)
            except requests.exceptions.HTTPError as e:
                if num_retries > 5:
                    raise e
                print(f"Retrying... {num_retries+1}")
                num_retries += 1
                time.sleep(10)     
        except Exception as e:
            print(e)
            if num_retries > 5:
                raise e
            print(f"Retrying... {num_retries+1}")   

    return r[0]["response"], mem_id


##### Embedding

In [4]:

url = "http://localhost:8000/embedding"
r = requests.post(url, json={"admin_password": "valory_algover"})
r.raise_for_status()
r.json()

{'ok': True, 'message': 'Embedding request received'}

##### chat

In [5]:
res, mem_id = get_answer("what is open autonomy")

Markdown(res)

Request sent. Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...


Open Autonomy is a framework for the creation of agent services. These agent services are off-chain autonomous services that run as a multi-agent system and offer enhanced functionalities on-chain. They expand the range of operations that traditional smart contracts provide, allowing for the execution of arbitrarily complex operations such as machine learning algorithms. These agent services are decentralized, trust-minimized, transparent, and robust. The framework provides command line tools and base packages to build, deploy, publish, and test agent services.

Based on the given context, Open Autonomy is a framework that enables the creation of decentralized and autonomous services that run off-chain but offer enhanced functionality on-chain. These services can execute complex operations and are built using command line tools and base packages provided by the framework.

In [6]:
res, mem_id = get_answer("What is an agent service")

Markdown(res)

Request sent. Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...


An agent service is a type of off-chain autonomous service that runs as a multi-agent system and is secured on a public blockchain. It is decentralized, trust-minimized, transparent, and robust. An agent service is composed of multiple agents and is owned by an operator who manages the infrastructure. Each agent within the service has its own set of functionalities and implements different communication protocols. The core component inside an agent is the FSM app, which defines the business logic of the service. The internal state of an agent service is replicated across all agents through a consensus gadget. Agent services can perform complex, proactive behaviors and can interact with smart contracts. They are different from smart contracts, web services, and custom decentralized services in terms of their architecture and capabilities.

In [7]:
res, mem_id = get_answer("why do we need agent services")

Markdown(res)

Request sent. Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...


Based on the provided context, agent services are needed because they go beyond simple, reactive applications like smart contracts and can exhibit complex, proactive behaviors that involve off-chain logic. They can trigger specific actions on external events and execute machine learning models. Agent services are decentralized and can provide functionalities to objects living on-chain. They are designed to be autonomous and can act as decentralized oracles for smart contracts or execute complex investing strategies that cannot be easily encoded on-chain. In summary, agent services provide advanced capabilities and functionalities that are not possible with other software solutions like smart contracts, web services, or custom decentralized services.

In [9]:
res, mem_id = get_answer("what are the key concepts of open autonomy")

Markdown(res)

Request sent. Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...


The key concepts of open autonomy are:

1. Agent services: These are off-chain autonomous services that run as a multi-agent-system (MAS) and offer enhanced functionalities on-chain. They expand the range of operations that traditional smart contracts provide, allowing for the execution of arbitrarily complex operations.

2. Finite-state machine (FSM) specification: The business logic of the service is described as a finite-state machine (FSM) specification. It defines the states of the service and how to transition between them.

3. FSM app: The actual business logic is encoded in the FSM app that lives inside each agent. The FSM app is responsible for executing the service's functionality.

4. Composability, extensibility, and reusability: Open autonomy emphasizes the advantages of composability, extensibility, and reusability, allowing developers to build on existing components and create new services by combining them.

5. Decentralization: Open autonomy is decentralized, meaning that the agent services and the ecosystem surrounding them operate in a decentralized manner. This includes decentralized governance, reward systems for developers and operators, and the ability to vote on network parameters.

6. Communication between agents: Agents communicate with each other using different forms of communication depending on the service status. Before the agents can establish a temporary blockchain (Tendermint network) that serves as a consensus mechanism.

Source: [Open Autonomy Documentation](https://github.com/valory-xyz/docs/blob/67f4e7878dc7094ab472b422ecc6af982f7c0246/open-autonomy/docs/index.md)

### Chat using memory

In [11]:
res, mem_id = get_answer("could you explain point two", mem_id)

Markdown(res)

Request sent. Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...
Waiting for response...


The description of the business logic of the service in point two is to describe the service's business logic as a finite-state machine (FSM) in a language understood by the framework. This specification defines the states of the service and how to transition from one state to another based on observed events.